<a href="https://colab.research.google.com/github/emaynes/Portfolio/blob/main/FinalFittedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression # to build a LR model for comparison
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization 
import statsmodels.api as sm # to build a LOWESS model
from scipy.interpolate import interp1d # for interpolation of new data points

from statsmodels.nonparametric.smoothers_lowess import lowess as lowess #Lowess model
from sklearn.model_selection import train_test_split

drive.mount('/content/gdrive')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Mounted at /content/gdrive


In [5]:
#Here I call in the data files from my google drive, which I attached 
weeklydata18 = pd.read_csv('/content/gdrive/My Drive/Old Coursework/488/london/d18.csv')
weeklydata18[['prcp','wspd','Value']].describe()
weeklydata19 = pd.read_csv('/content/gdrive/My Drive/Old Coursework/488/london/d19.csv')
weeklydata19[['prcp','wspd','Value']].describe()
weeklydata20 = pd.read_csv('/content/gdrive/My Drive/Old Coursework/488/london/d20.csv')
weeklydata20[['prcp','wspd','Value']].describe()

,prcp,wspd,Value
count,53.000000,53.000000,53.000000
mean,1.795553,13.827314,27.041159
std,1.948458,3.749340,7.754059
min,0.000000,3.985714,15.552620
25%,0.214286,11.571429,22.041408
50%,1.528571,13.242857,25.149430
75%,2.571429,16.257143,31.317378
max,10.871429,21.071429,49.824201


In [6]:
#Run a Linear Regression with 2019 and 2018 data, regressing the emissions levels on weather covariates

#x and y values for Linear Regression on 2018
X_train = weeklydata18[['prcp','wspd']].values
y_train =weeklydata18['Value'].values

#values for 2019
X_test = weeklydata19[['prcp','wspd']].values
y_test = weeklydata19['Value'].values

#values for 2020
X_result = weeklydata20[['prcp','wspd']].values
y_result = weeklydata20['Value'].values


# ------- Linear Regression -------
# Fit 2018 and 2019 data to the LR and recover the residuals
model1 = LinearRegression()
LR_1 = model1.fit(X_train, y_train)
y_train_predict = model1.predict(X_train)
print(y_train_predict)
resid_train = y_train - y_train_predict
print(resid_train)

model2 = LinearRegression()
LR_2 = model2.fit(X_test, y_test)
y_test_predict = model2.predict(X_test)
print(y_test_predict)
resid_test = y_test - y_test_predict
print(resid_test)

[36.59974865 38.46454619 33.90985912 37.47568365 36.85303515 37.56689807
 37.84873074 37.31012284 35.09176313 38.97397698 38.50864994 36.90297999
 44.15179638 38.40507718 43.29970756 38.86216488 35.501838   38.77531305
 40.00917282 38.55256137 43.63596779 41.14876536 38.52624407 35.9822381
 36.69745805 37.85076662 38.42974729 39.36839777 38.49992677 36.26838411
 39.98503648 39.70596645 36.99363635 37.57707612 42.27754958 38.03251594
 37.0077404  34.84981791 42.14111663 39.47570219 35.94787512 46.68402502
 38.127801   39.19857087 40.9698745  37.67556019 39.75884331 37.53767237
 35.29745445 38.89313494 36.44228143 41.45148722 38.71046519]
[ -1.26793355   5.78276428   6.76387248   2.24193877   6.4406243
   6.34252727  12.55585093   5.26956631   6.42366827  10.78036886
   0.68643689   5.39542637   1.32832737  -1.33636939   0.31413503
   8.95656238  -3.16674769   6.458147     5.02349354  -3.88712531
  -5.21545537  -4.03258503  -8.87319464  -5.98844585  -9.89456633
  -4.59794055   0.40344831

In [7]:
print("Training set score: {:.2f}".format(LR_1.score(X_train, y_train)))
print("Test set score: {:.2f}".format(LR_1.score(X_test, y_test)))

Training set score: 0.15
Test set score: -0.04


In [8]:
# ------- LOWESS -------
# Generate y_hat values using lowess and cross validate 
# using 2018 residuals as training and 2019 residuals as test set
x = weeklydata18['week'].values 

#Lowess with default parameters frac = 2/3
low_1 = lowess(resid_train, x) # note, default frac=2/3
low1 = pd.DataFrame(low_1)
low1 = low1.rename({0: 'week', 1: 'low1'}, axis='columns')
weeklydata2018_1 = weeklydata18.combine_first(low1)
weeklydata2019_1 = weeklydata19.combine_first(low1)

#Lowess with parameters frac = 1/5
low_2 = lowess(resid_train, x, frac=1/5)
low2 = pd.DataFrame(low_2)
low2 = low2.rename({0: 'week', 1: 'low2'}, axis='columns')
weeklydata2018_2 = weeklydata18.combine_first(low2)
weeklydata2019_2 = weeklydata19.combine_first(low2)

#Lowess with parameters frac = 1/4
low_3 = lowess(resid_train, x, frac=1/4)
low3 = pd.DataFrame(low_3)
low3 = low3.rename({0: 'week', 1: 'low3'}, axis='columns')
weeklydata2018_3 = weeklydata18.combine_first(low3)
weeklydata2019_3 = weeklydata19.combine_first(low3)

#Lowess with parameters frac = 1/3
low_4 = lowess(resid_train, x, frac=1/3)
low4 = pd.DataFrame(low_4)
low4 = low4.rename({0: 'week', 1: 'low4'}, axis='columns')
weeklydata2018_4 = weeklydata18.combine_first(low4)
weeklydata2019_4 = weeklydata19.combine_first(low4)

#Lowess with parameters frac = 1/2
low_5 = lowess(resid_train, x, frac=1/2)
low5 = pd.DataFrame(low_5)
low5 = low5.rename({0: 'week', 1: 'low5'}, axis='columns')
weeklydata2018_5 = weeklydata18.combine_first(low5)
weeklydata2019_5 = weeklydata19.combine_first(low5)

In [9]:
#Below is a manual form of cross validation for LOESS ...

In [10]:
x1 = weeklydata2018_1[['low1','prcp','wspd']]
y1 = weeklydata2018_1['Value']
x2 = weeklydata2019_1[['low1','prcp','wspd']]
y2 = weeklydata2019_1['Value']

LR1 = LinearRegression().fit(x1, y1)
print("Training set score: {:.2f}".format(LR1.score(x1, y1)))
print("Test set score: {:.2f}".format(LR1.score(x2, y2)))

Training set score: 0.56
Test set score: 0.59


In [11]:
x1 = weeklydata2018_2[['low2','prcp','wspd']]
y1 = weeklydata2018_2['Value']
x2 = weeklydata2019_2[['low2','prcp','wspd']]
y2 = weeklydata2019_2['Value']

LR1 = LinearRegression().fit(x1, y1)
print("Training set score: {:.2f}".format(LR1.score(x1, y1)))
print("Test set score: {:.2f}".format(LR1.score(x2, y2)))

Training set score: 0.72
Test set score: 0.65


In [12]:
x1 = weeklydata2018_3[['low3','prcp','wspd']]
y1 = weeklydata2018_3['Value']
x2 = weeklydata2019_3[['low3','prcp','wspd']]
y2 = weeklydata2019_3['Value']

LR1 = LinearRegression().fit(x1, y1)
print("Training set score: {:.2f}".format(LR1.score(x1, y1)))
print("Test set score: {:.2f}".format(LR1.score(x2, y2)))

Training set score: 0.70
Test set score: 0.66


In [13]:
x1 = weeklydata2018_4[['low4','prcp','wspd']]
y1 = weeklydata2018_4['Value']
x2 = weeklydata2019_4[['low4','prcp','wspd']]
y2 = weeklydata2019_4['Value']

LR1 = LinearRegression().fit(x1, y1)
print("Training set score: {:.2f}".format(LR1.score(x1, y1)))
print("Test set score: {:.2f}".format(LR1.score(x2, y2)))

Training set score: 0.66
Test set score: 0.66


In [14]:
x1 = weeklydata2018_5[['low5','prcp','wspd']]
y1 = weeklydata2018_5['Value']
x2 = weeklydata2019_5[['low5','prcp','wspd']]
y2 = weeklydata2019_5['Value']

LR1 = LinearRegression().fit(x1, y1)
print("Training set score: {:.2f}".format(LR1.score(x1, y1)))
print("Test set score: {:.2f}".format(LR1.score(x2, y2)))

Training set score: 0.58
Test set score: 0.62


In [15]:
#Results show that Lowess with frac = 1/3 is most accurate

In [16]:
#include the outcomes of the trained LOESS model in the 2020 data to run the linear regression
weeklydata2020 = weeklydata20.combine_first(low4)

In [17]:
weeklydata2020

,Value,date,day,lockdown_1,lockdown_2,low4,month,open,prcp,pres,week,weeks,wspd,year
0,32.860702,3.0,4.000000,0.000000,0.000000,4.414839,1.000000,1.000000,0.400000,1026.071429,0.0,0.285714,16.100000,2020.0
1,26.790330,10.0,11.000000,0.000000,0.000000,4.677341,1.000000,1.000000,1.785714,1013.857143,1.0,1.285714,20.271429,2020.0
2,49.824201,17.0,18.000000,0.000000,0.000000,4.898305,1.000000,1.000000,2.685714,1027.914286,2.0,2.285714,11.100000,2020.0
3,41.004106,24.0,25.000000,0.000000,0.000000,5.079062,1.000000,1.000000,1.757143,1016.614286,3.0,3.285714,12.071429,2020.0
4,26.251774,31.0,14.285714,0.000000,0.000000,5.218373,1.571429,1.000000,1.500000,1009.571429,4.0,4.285714,21.071429,2020.0
5,35.832509,38.0,8.000000,0.000000,0.000000,5.318433,2.000000,1.000000,2.542857,1014.557143,5.0,5.285714,13.085714,2020.0
6,25.218852,45.0,15.000000,0.000000,0.000000,5.382607,2.000000,1.000000,5.185714,1008.500000,6.0,6.285714,19.828571,2020.0
7,23.619944,52.0,22.000000,0.000000,0.000000,5.411294,2.000000,1.000000,2.071429,1014.185714,7.0,7.285714,20.528571,2020.0
8,31.670695,59.0,16.571429,0.000000,0.000000,5.333870,2.428571,1.000000,4.000000,997.342857,8.0,8.285714,19.700000,2020.0
9,32.124650,66.0,7.000000,0.000000,0.000000,4.876337,3.000000,1.000000,3.857143,1006.428571,9.0,9.285714,17.028571,2020.0


In [18]:
#linear regression of 2020 data on weather variables and time trend model
X = weeklydata2020[['low4','prcp','wspd','lockdown_1','lockdown_2']]
y = weeklydata2020['Value']

LinearReg = LinearRegression().fit(X,y)
print("score: {:.2f}".format(LinearReg.score(X, y)))

print('coefficient:' , LinearReg.coef_)
print('intercept:', LinearReg.intercept_)

score: 0.72
coefficient: [ 1.48602857  0.11637028 -1.48930638 -9.1436743   0.37686912]
intercept: 48.97876656179211


In [19]:
#output in a different form
import statsmodels.api as sm
X = sm.add_constant(X)
ols = sm.OLS(y.values, X)
ols_result = ols.fit()
ols_result.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.724
Model:                            OLS   Adj. R-squared:                  0.695
Method:                 Least Squares   F-statistic:                     24.66
Date:                Sat, 26 Feb 2022   Prob (F-statistic):           4.25e-12
Time:                        00:01:14   Log-Likelihood:                -149.14
No. Observations:                  53   AIC:                             310.3
Df Residuals:                      47   BIC:                             322.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         48.9788      2.676     18.306      0.000      43.596      54.361
low4           1.4860      0.174      8.559      0.000       1.137       1.835
prcp           0.1164      0.334      0.349      0.729      -0.555       0.787
wspd          -1.4893      0.187     -7.972      0.000      -1.865      -1.113
lockdown_1    -9.1437      1.957     -4.672      0.000     -13.081      -5.206
lockdown_2     0.3769      2.454      0.154      0.879      -4.560       5.314
==============================================================================
Omnibus:                        0.310   Durbin-Watson:                   1.385
Prob(Omnibus):                  0.856   Jarque-Bera (JB):                0.053
Skew:                           0.072   Prob(JB):                        0.974
Kurtosis:                       3.056   Cond. No.                         76.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
#prepare the data for the graph
data = [weeklydata18["Value"], weeklydata19['Value'], weeklydata20["Value"],weeklydata2020["week"]]
headers = ["2018", "2019","2020","week"]
plotdata = pd.concat(data, axis=1, keys=headers)
plotdata

plotdata = plotdata.rename({0: '2018', 1: '2019', 2:"2020", 3:"week"}, axis='columns')
plotdata

,2018,2019,2020,week
0,35.331815,43.004033,32.860702,0.0
1,44.247310,41.036060,26.790330,1.0
2,40.673732,48.658716,49.824201,2.0
3,39.717622,47.457616,41.004106,3.0
4,43.293659,47.584590,26.251774,4.0
5,43.909425,34.637623,35.832509,5.0
6,50.404582,53.091784,25.218852,6.0
7,42.579689,52.873901,23.619944,7.0
8,41.515431,49.997213,31.670695,8.0
9,49.754346,28.631866,32.124650,9.0


In [76]:
#generate the graph with year 2018 -- 2020 and the LOESS model for seasonality
fig = px.line(plotdata, x=plotdata['week'], y=plotdata['2020'], color_discrete_sequence=['black'],labels={
                     "week": "Week Index",
                     "2020": "NO_2 Concentration Levels (ug/m^3)"})
fig.update_layout(dict(plot_bgcolor = 'white'))

fig.add_scatter(x=plotdata['week'], y=plotdata['2020'], mode='lines', name='2020', line=dict(color='rgb(131, 90, 241)'))
fig.add_scatter(x=plotdata['week'], y=plotdata['2019'], mode='lines', name='2019', line=dict(color='rgb(127, 166, 238)'), line_dash="dash")
fig.add_scatter(x=plotdata['week'], y=plotdata['2018'], mode='lines', name = '2018', line=dict(color='rgb(179, 197, 255)'), line_dash="dash")

fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black')

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black')

fig.add_trace(go.Scatter(x=[11.15,11.15], y=[17,52], mode="lines", line_width=0, showlegend = False, name = 'lockdown 1 start', line=dict(color='rgb(179, 197, 255)')))
fig.add_trace(go.Scatter(x=[18.71,18.71], y=[17,52], fill='tonexty', mode="lines", line_width=0, name = 'Lockdown Periods', line=dict(color='rgb(179, 197, 255)')))
fig.add_trace(go.Scatter(x=[43.15,43.15], y=[17,52], mode="lines", line_width=0, showlegend = False, name = 'lockdown 2 start', line=dict(color='rgb(179, 197, 255)')))
fig.add_trace(go.Scatter(x=[48,48], y=[17,52], fill='tonexty', mode="lines", line_width=0, showlegend = False, name = 'lockdown 2', line=dict(color='rgb(179, 197, 255)')))

fig.add_traces(go.Scatter(x=low_2[:,0], y=low_2[:,1], name='LOWESS', line=dict(color='rgb(204,102,102)')))

fig.show() 